In [1]:
from keras import backend as K
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.applications import ResNet50
import numpy as np
import pandas as pd 
from keras.utils.data_utils import get_file
import matplotlib.pyplot as plt
import requests
import os
import importlib
import numpy as np
import cv2
from PIL import Image , ImageFilter

Using TensorFlow backend.


## Crop face with opencv

In [ ]:
### Cropping faces
in_folder = "./SCUT-FBP5500_v2/Images/"
out_folder = "./SCUT-FBP5500_v2/cropped_Images_close/"

onlyfiles = [f for f in os.listdir(in_folder) if os.path.isfile(os.path.join(in_folder, f))]
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')


for file in onlyfiles:
    
    image = cv2.imread(in_folder+file)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        
      print("No faces found")
      print(file)
      cv2.imwrite(out_folder+file,image)

    else:
        for (x,y,w,h) in faces:
#             x_start = y - bounding_offset  if y - bounding_offset >= 0  else y
#             x_end = y+h+bounding_offset  if y+h+bounding_offset <= image.shape[0]  else y+h
#             y_start =  x - bounding_offset if x - bounding_offset >= 0 else x
#             y_end = x+w+bounding_offset  if x+w+bounding_offset <= image.shape[1] else x+w
            cropped_img = image[x:x+w, y:y + h, :]    
#             cropped_img = image[y - bounding_offset:y+h+bounding_offset,x - bounding_offset:x+w+bounding_offset,:]
            cv2.imwrite(out_folder+file,cropped_img)

# Data Augmentation

In [19]:
# Augmentation
import random 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


def blur_at_random(img_array):    
    img =  Image.fromarray(img_array.astype('uint8'))
    blurred_image = img.filter(ImageFilter.BoxBlur(random.randint(1,4)))
    result = np.array(blurred_image)
    return result 

def generate_img(filename):
    img = load_img(in_folder + filename)    
    img.save(out_folder+filename)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=out_folder, 
                              save_prefix=filename.split(".")[0],
                              save_format='jpg'):
        break

in_folder = "./SCUT-FBP5500_v2/cropped_Images/"
out_folder = "./SCUT-FBP5500_v2/augmented/"


datagen = ImageDataGenerator(
    preprocessing_function=blur_at_random,
    rotation_range=20,
    channel_shift_range = 50,
    horizontal_flip=True
    )

onlyfiles = [f for f in os.listdir(in_folder) if os.path.isfile(os.path.join(in_folder, f))]


for file in onlyfiles:
    generate_img(file)
    

# Loading train_x

In [ ]:
### Loading training images into memory 


folder = "./SCUT-FBP5500_v2/augmented/"
onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

train_files = []
y_train = []

i=0

for _file in onlyfiles:
    train_files.append(_file)   
    y_train.append(_file.title())

print("Files in train_files: %d" % len(train_files))

image_width = 350
image_height = 350
channels = 3


dataset = np.ndarray(shape=(len(train_files), image_width, image_height,  channels),
                     dtype=np.float32)


i = 0
for _file in train_files:
    img = load_img(folder + "/" + _file,target_size=(350, 350))  # this is a PIL image
    #img = load_img(folder + "/" + _file,target_size=(224, 224))
    x = img_to_array(img)  
    x = x  / 255.0
    dataset[i] = x
    i = i + 1
    if i % 250 == 0:
         print("%d images to array" % i)

print("All images to array!")


# prepare train_y (rating)

In [22]:
df = pd.read_excel("./SCUT-FBP5500_v2/All_Ratings.xlsx")
df = df.groupby(["Filename"])["Rating"].mean()
df = df.to_frame()
df.to_csv('ratings.csv')
df = pd.read_csv('ratings.csv',index_col='Filename')


for i in range(len(y_train)):
    transformed = y_train[i].split(".")
    transformed[0] = transformed[0].upper()
    transformed[0] = transformed[0].split("_")[0]
    transformed[1] = transformed[1].lower()
    transformed = ".".join(transformed)
    y_train[i] = transformed

y_train_label = []
for file_name in y_train:
    target_row = df.loc[file_name]
    y_train_label.append(target_row['Rating'])
    
    
y_train_label =  np.array(y_train_label)
y_train_label = y_train_label.reshape((16500,1))
    

# Training

In [23]:
resnet = ResNet50(include_top=False, pooling='avg')
model = Sequential()
model.add(resnet)
model.add(Dense(1))
model.layers[0].trainable = False
print(model.summary()) 
model.compile(loss='mean_squared_error', optimizer=Adam())

In [24]:
model.fit(batch_size=32, x=dataset, y=y_train_label, epochs=30)